In [1]:
import os
import time
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
# config
DATA_DIR = '/kaggle/input/dataset-batik-indonesia'
TRAIN_DIR = os.path.join(DATA_DIR, 'preprocessing_images')
VAL_DIR = os.path.join(DATA_DIR, 'dataset_split/val')
TEST_DIR = os.path.join(DATA_DIR, 'dataset_split/test')

OUTPUT_DIR = './batik_results'
os.makedirs(OUTPUT_DIR, exist_ok=True)

BATCH_SIZE = 32
NUM_EPOCHS = 50
LR = 0.001
PATIENCE = 5
MODEL_NAMES = [
    'resnet18', 'resnet34', 'resnet50',
    'resnet101', 'resnet152', 'resnet110'
]
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# data transform
basic_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [4]:
# data loaders
def get_loaders(train_dir, val_dir, test_dir, batch_size):
    image_datasets = {
        'train': datasets.ImageFolder(train_dir, basic_transforms),
        'val': datasets.ImageFolder(val_dir, basic_transforms),
        'test': datasets.ImageFolder(test_dir, basic_transforms)
    }
    dataloaders = {
        x: DataLoader(image_datasets[x], batch_size=batch_size,
                      shuffle=(x == 'train'), num_workers=2)
        for x in ['train', 'val', 'test']
    }
    return dataloaders, image_datasets['train'].classes

loaders, class_names = get_loaders(TRAIN_DIR, VAL_DIR, TEST_DIR, BATCH_SIZE)
NUM_CLASSES = len(class_names)

In [5]:
# init model
from torchvision.models.resnet import ResNet, BasicBlock

def initialize_model(name, num_classes, pretrained=True):
    if name == 'resnet18': model = models.resnet18(pretrained=pretrained)
    elif name == 'resnet34': model = models.resnet34(pretrained=pretrained)
    elif name == 'resnet50': model = models.resnet50(pretrained=pretrained)
    elif name == 'resnet101': model = models.resnet101(pretrained=pretrained)
    elif name == 'resnet152': model = models.resnet152(pretrained=pretrained)
    elif name == 'resnet110':  # Custom ResNet-110
        model = ResNet(BasicBlock, [18, 18, 18], num_classes=num_classes)
    else:
        raise ValueError(f"Unknown model {name}")

    if hasattr(model, 'fc'):
        in_features = model.fc.in_features
        model.fc = nn.Linear(in_features, num_classes)

    return model.to(DEVICE)

In [6]:
# training
def train_model(model, dataloaders, criterion, optimizer, num_epochs, patience):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}
    counter = 0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()
            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    preds = outputs.argmax(dim=1)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            history[f"{phase}_loss"].append(epoch_loss)
            history[f"{phase}_acc"].append(epoch_acc.item())

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            # early stopping check
            if phase == 'val':
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    counter = 0
                else:
                    counter += 1
                    if counter >= patience:
                        print("Early stopping.")
                        model.load_state_dict(best_model_wts)
                        return model, history, best_acc.item()
    model.load_state_dict(best_model_wts)
    return model, history, best_acc.item()

In [7]:
# evaluation
def evaluate_model(model, dataloader):
    model.eval()
    preds_all, labels_all = [], []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(DEVICE)
            outputs = model(inputs)
            preds = outputs.argmax(dim=1).cpu().numpy()
            preds_all.extend(preds)
            labels_all.extend(labels.numpy())
    return (
        accuracy_score(labels_all, preds_all),
        precision_score(labels_all, preds_all, average='macro'),
        recall_score(labels_all, preds_all, average='macro'),
        f1_score(labels_all, preds_all, average='macro'),
        confusion_matrix(labels_all, preds_all)
    )

In [8]:
# plotting
def plot_history(history, model_name):
    epochs = range(1, len(history['train_loss']) + 1)
    plt.figure()
    plt.plot(epochs, history['train_loss'], label='Train Loss')
    plt.plot(epochs, history['val_loss'], label='Val Loss')
    plt.title(f'{model_name} - Loss')
    plt.legend()
    plt.savefig(os.path.join(OUTPUT_DIR, f'{model_name}_loss.png'))
    plt.close()

    plt.figure()
    plt.plot(epochs, history['train_acc'], label='Train Acc')
    plt.plot(epochs, history['val_acc'], label='Val Acc')
    plt.title(f'{model_name} - Accuracy')
    plt.legend()
    plt.savefig(os.path.join(OUTPUT_DIR, f'{model_name}_acc.png'))
    plt.close()

In [ ]:
# main
results = []
best_model_name = ''
best_overall_acc = 0.0
for model_name in MODEL_NAMES:
    print(f"\n--- Training {model_name} ---")
    model = initialize_model(model_name, NUM_CLASSES)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LR)

    trained_model, hist, val_acc = train_model(model, loaders, criterion, optimizer, NUM_EPOCHS, PATIENCE)
    acc, prec, rec, f1, cm = evaluate_model(trained_model, loaders['test'])

    torch.save(trained_model.state_dict(), os.path.join(OUTPUT_DIR, f"{model_name}.pth"))

    results.append({
        'Model': model_name,
        'Val Accuracy': val_acc,
        'Test Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1': f1
    })

    if val_acc > best_overall_acc:
        best_overall_acc = val_acc
        best_model_name = model_name

    plot_history(hist, model_name)


--- Training resnet18 ---


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 181MB/s]


Epoch 1/50
train Loss: 1.3787 Acc: 0.6017
val Loss: 1.5955 Acc: 0.5833
Epoch 2/50
train Loss: 0.7239 Acc: 0.7817
val Loss: 1.3280 Acc: 0.6667
Epoch 3/50
train Loss: 0.4272 Acc: 0.8643
val Loss: 1.1391 Acc: 0.7067
Epoch 4/50
train Loss: 0.3504 Acc: 0.8928
val Loss: 1.0256 Acc: 0.7367
Epoch 5/50
train Loss: 0.2266 Acc: 0.9296
val Loss: 1.4441 Acc: 0.6633
Epoch 6/50
train Loss: 0.2704 Acc: 0.9127
val Loss: 1.0903 Acc: 0.7167
Epoch 7/50
train Loss: 0.1806 Acc: 0.9469
val Loss: 0.8693 Acc: 0.8033
Epoch 8/50
train Loss: 0.0840 Acc: 0.9734
val Loss: 1.0181 Acc: 0.8033
Epoch 9/50
train Loss: 0.1524 Acc: 0.9532
val Loss: 1.6451 Acc: 0.7400
Epoch 10/50
train Loss: 0.1206 Acc: 0.9638
val Loss: 1.1651 Acc: 0.7767
Epoch 11/50


In [ ]:
# save summary
df = pd.DataFrame(results)
df.to_csv(os.path.join(OUTPUT_DIR, 'summary.csv'), index=False)
print("\nBest Model:", best_model_name)
print(df)